In [10]:
!pip install torchbearer
import torch
from torchvision import transforms
from torch.utils.data import Dataset
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchbearer
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchbearer import Trial
class MyDataset(Dataset):
  def __init__(self, size=5000, dim=40, random_offset=0):
        super(MyDataset, self).__init__()
        self.size = size
        self.dim = dim
        self.random_offset = random_offset

  def __getitem__(self, index):
      if index >= len(self):
          raise IndexError("{} index out of range".format(self.__class__.__name__))

      rng_state = torch.get_rng_state()
      torch.manual_seed(index + self.random_offset)

      while True:
        img = torch.zeros(self.dim, self.dim)
        dx = torch.randint(-10,10,(1,),dtype=torch.float)
        dy = torch.randint(-10,10,(1,),dtype=torch.float)
        c = torch.randint(-20,20,(1,), dtype=torch.float)

        params = torch.cat((dy/dx, c))
        xy = torch.randint(0,img.shape[1], (20, 2), dtype=torch.float)
        xy[:,1] = xy[:,0] * params[0] + params[1]

        xy.round_()
        xy = xy[ xy[:,1] > 0 ]
        xy = xy[ xy[:,1] < self.dim ]
        xy = xy[ xy[:,0] < self.dim ]

        for i in range(xy.shape[0]):
          x, y = xy[i][0], self.dim - xy[i][1]
          img[int(y), int(x)]=1
        if img.sum() > 2:
          break

      torch.set_rng_state(rng_state)
      return img.unsqueeze(0), params

  def __len__(self):
      return self.size

train_data = MyDataset()
val_data = MyDataset(size=500, random_offset=33333)
test_data = MyDataset(size=500, random_offset=99999)

In [0]:
trainloader = DataLoader(train_data, batch_size=128, shuffle=True)
validloader = DataLoader(val_data, batch_size=128, shuffle=True)
testloader = DataLoader(test_data, batch_size=128, shuffle=True)

In [0]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 48, (3,3),stride=1, padding=1)
        self.fc1 = nn.Linear(48*40**2, 128)
        self.fc2 = nn.Linear(128, 2)
            
    def forward(self, x):
        out = self.conv1(x)
        out = F.relu(out)
        # out = F.dropout(out, 0.2)
        out = out.view(out.shape[0], -1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        return out

In [0]:
# build the model
model = SimpleCNN()

# define the loss function and the optimiser
loss_function = nn.MSELoss()
optimiser = optim.Adam(model.parameters())

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader, test_generator=testloader)
trial.run(epochs=100)
results = trial.evaluate(data_key=torchbearer.TEST_DATA)
print(results)



{'test_loss': 13.315376281738281, 'test_mse': 13.361611366271973}


In [12]:
class SimpleCNN2(nn.Module):
    def __init__(self):
        super(SimpleCNN2, self).__init__()
        self.conv1 = nn.Conv2d(1, 48, (3,3),stride=1, padding=1)
        self.fc1 = nn.Linear(960, 128)
        self.fc2 = nn.Linear(128, 2)
        self.gp1 = nn.AdaptiveAvgPool2d((20,1)) 
    def forward(self, x):
        out = self.conv1(x)
        out = F.relu(out)
        out = self.conv1(x)
        out = F.relu(out)
        out = self.gp1(out)
        # out = F.dropout(out, 0.2)
        out = out.view(out.shape[0], -1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        return out

model = SimpleCNN2()

# define the loss function and the optimiser
loss_function = nn.MSELoss()
optimiser = optim.Adam(model.parameters())

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader, test_generator=testloader)
trial.run(epochs=100)
results = trial.evaluate(data_key=torchbearer.TEST_DATA)
print(results)


KeyboardInterrupt: ignored

In [0]:
class SimpleCNN2(nn.Module):
    def __init__(self):
        super(SimpleCNN2, self).__init__()
        self.conv1 = nn.Conv2d(1, 48, (3,3),stride=1, padding=1)
        self.fc1 = nn.Linear(960*20, 128)
        self.fc2 = nn.Linear(128, 2)
        self.gp1 = nn.AdaptiveAvgPool2d((20,20)) 
    def forward(self, x):
        out = self.conv1(x)
        out = F.relu(out)
        out = self.conv1(x)
        out = F.relu(out)
        out = self.gp1(out)
        # out = F.dropout(out, 0.2)
        out = out.view(out.shape[0], -1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        return out

model = SimpleCNN2()

# define the loss function and the optimiser
loss_function = nn.MSELoss()
optimiser = optim.Adam(model.parameters())

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader, test_generator=testloader)
trial.run(epochs=100)
results = trial.evaluate(data_key=torchbearer.TEST_DATA)
print(results)


{'test_loss': 8.248275756835938, 'test_mse': 8.278209686279297}


In [17]:
class SimpleCNN3(nn.Module):
    def __init__(self):
        super(SimpleCNN3, self).__init__()
        self.conv1 = nn.Conv2d(3, 48, (3,3),stride=1, padding=1)
        self.conv2 = nn.Conv2d(48, 48, (3,3),stride=1, padding=1)
        self.fc1 = nn.Linear(960*20, 128)
        self.fc2 = nn.Linear(128, 2)
        self.gp1 = nn.AdaptiveAvgPool2d((20,20)) 
    def forward(self, x):

        idxx = torch.repeat_interleave(torch.arange(-20,20,dtype=torch.float).unsqueeze(0)/40.0,repeats=40,dim=0).to(x.device)
        idxy = idxx.clone().t()
        idx =  torch.stack([idxx,idxy]).unsqueeze(0)
        idx = torch.repeat_interleave(idx,repeats=x.shape[0],dim=0)
        x = torch.cat([x,idx],dim=1)  
        out = self.conv1(x)
        out = F.relu(out)
        out = self.conv2(out)
        out = F.relu(out)
        out = self.gp1(out)
        out = out.view(out.shape[0], -1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        return out

model = SimpleCNN3()

# define the loss function and the optimiser
loss_function = nn.MSELoss()
optimiser = optim.Adam(model.parameters())

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader, test_generator=testloader)
trial.run(epochs=100)
results = trial.evaluate(data_key=torchbearer.TEST_DATA)
print(results)


KeyboardInterrupt: ignored

In [8]:
idxx = torch.repeat_interleave(torch.arange(-20,20,dtype=torch.float).unsqueeze(0)/40.0,repeats=40,dim=0)
idxy = idxx.clone().t()
idx =  torch.stack([idxx,idxy]).unsqueeze(0)
#idx = torch.repeat_interleave(idx,repeats=x.shape[0],dim=0)
# x = torch.cat([x,idx],dim=1) 
print(idx)
print(idx.shape)


128*3*40*40
128*1*40*40

tensor([[[[-0.5000, -0.4750, -0.4500,  ...,  0.4250,  0.4500,  0.4750],
          [-0.5000, -0.4750, -0.4500,  ...,  0.4250,  0.4500,  0.4750],
          [-0.5000, -0.4750, -0.4500,  ...,  0.4250,  0.4500,  0.4750],
          ...,
          [-0.5000, -0.4750, -0.4500,  ...,  0.4250,  0.4500,  0.4750],
          [-0.5000, -0.4750, -0.4500,  ...,  0.4250,  0.4500,  0.4750],
          [-0.5000, -0.4750, -0.4500,  ...,  0.4250,  0.4500,  0.4750]],

         [[-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.4750, -0.4750, -0.4750,  ..., -0.4750, -0.4750, -0.4750],
          [-0.4500, -0.4500, -0.4500,  ..., -0.4500, -0.4500, -0.4500],
          ...,
          [ 0.4250,  0.4250,  0.4250,  ...,  0.4250,  0.4250,  0.4250],
          [ 0.4500,  0.4500,  0.4500,  ...,  0.4500,  0.4500,  0.4500],
          [ 0.4750,  0.4750,  0.4750,  ...,  0.4750,  0.4750,  0.4750]]]])
torch.Size([1, 2, 40, 40])
